# GAWS 明文实现

主要包括以下算法：

- SNP过滤
- PCA
- 优势比
- 趋势卡方计算（检验）

## 1. SNP过滤（略）

---

## 2. PCA

PCA步骤（设有$m$条$n$维数据）：

1. 将原始数据按列组成$n$行$m$列矩阵$X$
2. 将X的每一行（代表一个属性字段）进行零均值化，即减去这一行的均值
3. 求出协方差矩阵
4. 求出协方差矩阵的特征值及对应的特征向量$r$
5. 将特征向量按对应特征值大小从上到下按行排列成矩阵，取前$k$行组成矩阵$P$
6. 即为降维到$k$维后的数据

### 例:

如下，有10条样本，每条样本都有5个特征值（即矩阵$X$的大小为： m=10行 n=5列）

In [68]:
import numpy as np
import pandas as pd
from sklearn import decomposition

# 读入数据
data_file = './data/pca-data.txt'
data = pd.read_csv(data_file, sep='\t', index_col=False)
data.set_index('Unnamed: 0', inplace=True)
data.iloc[:10, 1:6]

,rs11252546,rs7909677,rs10904494,rs11591988,rs4508132
Unnamed: 0,,,,,
NA18532,0,0,0,1,1
NA18605,0,0,0,0,0
NA18542,0,0,0,2,2
NA18550,0,0,0,0,0
NA18608,0,1,0,1,1
NA18564,0,0,0,1,1
NA18571,0,0,0,0,1
NA18620,0,0,0,1,1
NA18623,0,0,0,0,0


---
- 调用sklearn库实现的PCA算法，其中：
  - `n_compoents=2`为取前2个最大的主成分。（指定降维后的维数为2）
  - 该库实现的PCA算法，默认输入的数据中，每一行代表一个样本，因此直接调用`pca.fit()`即可

In [72]:
pca = decomposition.PCA(n_components=2)
pca.fit(data.iloc[:10, 1:6])  # 每一行代表一个样本
ex_ratio = pca.explained_variance_ratio_  # 每个主成分占方差比例

print("主成分占方差比例(ex_ratio): {},\n主成分之和(fit_ratio): {}\n".format(ex_ratio, sum(ex_ratio)))
print("主成分分组(components) : shape: {}".format(pca.components_.shape))
pd.DataFrame(pca.components_)

主成分占方差比例(ex_ratio): [0.86034351 0.09301912],
主成分之和(fit_ratio): 0.9533626346432376

主成分分组(components) : shape: (2, 5)


,0,1,2,3,4
0,-0.0,0.068890,-0.0,0.719585,0.690979
1,-0.0,0.987676,-0.0,0.048374,-0.148847


从上面可以看到，主成分分组是一个$2\times5$的矩阵($U$)，如要进行降为操作，计算$Ux$即可。
如：

In [47]:
x = np.array([0, 1, 2, 1, 0])  # 输入数据大小为5×n，此处n=1
out_x = np.dot(pca.components_, x)  # 输出数据大小为2×n，此处n=1

print(out_x)

[0.78847498 1.03605036]


因此可以计算得到，原始数据的降为结果为：

In [69]:
pd.DataFrame(pca.transform(data.iloc[:10, 1:6]))

,0,1
0,0.488239,-0.124072
1,-0.922325,-0.023599
2,1.898802,-0.224545
3,-0.922325,-0.023599
4,0.557129,0.863604
5,0.488239,-0.124072
6,-0.231346,-0.172446
7,0.488239,-0.124072
8,-0.922325,-0.023599
9,-0.922325,-0.023599


---
- 下面为自主实现的PCA算法（基于numpy），根据上面描述的PCA的步骤，实现PCA的函数如下：

In [53]:
def my_pca(data_mat, topNfeat=9999999):
    mean_vals = np.mean(data_mat, axis=0)                 # 求均值
    mean_removed = data_mat - mean_vals                   # 2. 去平均值

    cov_mat = np.cov(mean_removed, rowvar=0)              # 3. 求协方差矩阵(若rowvar非0，一列代表一个样本)

    eig_vals, eig_vects = np.linalg.eig(np.mat(cov_mat))  # 求特征值&特征向量

    eig_val_ind = np.argsort(eig_vals)                    # 从小到大对N个值排序，返回其索引
    eig_val_ind = eig_val_ind[:-(topNfeat+1):-1]
    red_eig_vects = eig_vects[:, eig_val_ind]

    low_d_data_mat = np.dot(mean_removed, red_eig_vects)  # 将数据转换到新空间
    # 重构数据
    recon_mat = (low_d_data_mat * red_eig_vects.T) + mean_vals[:, np.newaxis].T

    # 返回PCA降维后的结果，重构的结果，以及主成分
    return low_d_data_mat, recon_mat, red_eig_vects

In [71]:
low_d_data_mat, recon_mat, red_eig_vects = my_pca(data.iloc[:10, 1:6], topNfeat=2)

print("主成分： shape: {}".format(red_eig_vects.shape))
pd.DataFrame(red_eig_vects)

主成分： shape: (5, 2)


,0,1
0,0.000000,0.000000
1,0.068890,-0.987676
2,0.000000,0.000000
3,0.719585,-0.048374
4,0.690979,0.148847


可以看到，主成分分组$U$的维度为($5\times2$)，因此计算$xU$的结果则为降为后的结果$((10\times5)\times(5\times2))$

同时，该计算结果与上面直接调用`sklearn`库的结果是一样的(只是$x$放的位置不一样) 

*注：不同的PCA算法实现的代码规范可能不一样，因此具体实现时需要先进行验算*

接下来计算降为后的结果和重构的结果

In [66]:
# PCA降维结果
low_d_data_mat = pd.DataFrame(low_d_data_mat)
low_d_data_mat

,0,1
0,0.488239,0.124072
1,-0.922325,0.023599
2,1.898802,0.224545
3,-0.922325,0.023599
4,0.557129,-0.863604
5,0.488239,0.124072
6,-0.231346,0.172446
7,0.488239,0.124072
8,-0.922325,0.023599
9,-0.922325,0.023599


In [67]:
# 重构后的数据
recon_mat = pd.DataFrame(recon_mat)
recon_mat

,0,1,2,3,4
0,0.0,0.011092,0.0,0.945327,1.055830
1,0.0,0.013153,0.0,-0.064833,0.066206
2,0.0,0.009030,0.0,1.955488,2.045455
3,0.0,0.013153,0.0,-0.064833,0.066206
4,0.0,0.991342,0.0,1.042677,0.956419
5,0.0,0.011092,0.0,0.945327,1.055830
6,0.0,-0.086258,0.0,0.425185,0.565813
7,0.0,0.011092,0.0,0.945327,1.055830
8,0.0,0.013153,0.0,-0.064833,0.066206
9,0.0,0.013153,0.0,-0.064833,0.066206


---

## 3. 优势比

|SNP      | 碱基A | 碱基T |
|:-------:|:----:|:-----:|
|Case组   |a     |b      |
|Control组|c     |d      |

对于次要等位碱基T来说：$T = \frac{bc}{da}$

OR 一般用于计算次要等位基因：
- OR值=1，表示该因素对疾病的发生不起作用。
- OR值>1，表示该因素是一个危险因素
- OR值<1，表示该因素是一个保护因素

## 4. 趋势卡方计算（检验）

### 原理：

如下面$2\times3$的列联表。其中，$R_1=N_{11}+N_{12}+N_{13}$，$C_1=N_{11}+N_{21}$，etc.

|         |B=1      |B=2      |B=3      |sum      |
|:-------:|:-------:|:-------:|:-------:|:-------:|
|A=1      |$N_{11}$ |$N_{12}$ |$N_{13}$ |$R_1$    |
|A=2      |$N_{21}$ |$N_{22}$ |$N_{23}$ |$R_2$    |
|sum      |$C_1$    |$C_2$    |$C_3$    |$N$      |

首先计算：①$T=\sum_{i=1}^k t_{i}(N_{1i}R_2 - N_{2i}R_1)$

然后计算②方差: $Var(T) = \frac{R_1 R_2}{N}(\sum_{i=1}^{k} t_i^2 C_i(N-C_i) - 2\sum_{i=1}^{k-1}\sum_{j=i+1}^{k}t_i t_j C_i C_j)$

最后计算: ③  $\frac{T}{\sqrt{Var(T)}}$

注：对于上面权重 $t$ 的选择，GWAS中一般选择$(1, 1, 0)$ / $(0, 1, 1)$ / $(0, 1, 2)$

### 例子:

假设有如下列联表：

|         | Genotype aa | Genotype Aa | Genotype AA | Sum |
|:-------:|:-----------:|:-----------:|:-----------:|:---:|
|Controls | 20          | 20          | 20          |60   |
|Cases    | 10          | 20          | 30          |60   |
|Sum      | 30          | 40          | 50          |120  |

计算得到：
- T = 600
- Var(T) = 105000.0
- score = 1.85...

*详见以下程序实现*

In [2]:
import math
import numpy as np
class CA_test_for_trend:
    def __init__(self, n_1, n_2, k=3, t=[1, 1, 0]):
        assert len(n_1) == k and len(n_2) == k, "len(n) must equals to k"
        assert len(t) == k, "len(k) must equals to k"
        self.k = k
        self.t = t
        self.table = np.array([n_1, n_2])
        self.R = self.table.sum(axis=1)
        self.C = self.table.sum(axis=0)
        self.N = self.table.sum()
    
    # 计算 T 值
    def cal_T(self):
        T = 0
        for i in range(self.k):
            T += self.t[i] * ( self.table[0][i] * self.R[1] - self.table[1][i] * self.R[0])
        return T

    # 计算 T 的方差
    def cal_varT(self):
        part_1 = self.R[0] * self.R[1] / self.N
        part_2_left = 0
        for i in range(self.k):
            part_2_left += self.t[i]**2 * self.C[i] * (self.N - self.C[i])

        part_2_right = 0
        for i in range(self.k):
            for j in range(i+1, self.k):
                part_2_right += self.t[i] * self.t[j] * self.C[i] * self.C[j]
        else:
            part_2_right *= 2

        return part_1 * (part_2_left - part_2_right)
    
    def cal_score(self, T, var_T):
        return T / math.sqrt(var_T)

# ex:
Controls = [20, 20, 20]
Cases = [10, 20, 30]
K = 3
t = [1, 1, 0]
myca = CA_test_for_trend(Controls, Cases, K, t)
T = myca.cal_T()
var_T = myca.cal_varT()
score = myca.cal_score(T, var_T)
print("T: {}\nVar(T): {}\nscore: {}".format(T, var_T, score))

T: 600
Var(T): 105000.0
score: 1.8516401995451028
